In [479]:
##importing Basic libraries
import pandas as pd
import pickle
from pandas import DataFrame, Series
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model, naive_bayes
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
import bokeh as bk
from sklearn.neighbors import KNeighborsRegressor
from matplotlib import pyplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error,r2_score
import plotly
import chart_studio.plotly as py
from chart_studio.plotly import plot, iplot 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
import pybaseball as pyb
from sklearn.linear_model import ElasticNet
from pybaseball import statcast
from pybaseball import statcast_pitcher
from pybaseball import statcast_batter
from pybaseball import statcast_pitcher_exitvelo_barrels
from pybaseball import statcast_batter_exitvelo_barrels
from pybaseball import statcast_batter_expected_stats
from pybaseball import statcast_pitcher_expected_stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pwlf as pwlf
#import matplotlib.backends
import matplotlib as mpl
import joblib
import pickle as pkl

In [480]:
#from test import keras
from keras.models import Sequential
#dense
from keras.layers import Dense
#adam
from keras.optimizers import Adam

In [481]:
df_2022 = pd.read_csv("/Users/johndavis/Desktop/Desktop_2/Statcast_ALL_TIME.csv",low_memory=False)
df_2023 = pd.read_csv("/Users/johndavis/Desktop/Desktop_2/MLB_2023_FULL.csv",low_memory=False)

In [482]:
#multiply pfx_z	pfx_x by 12 to get inches
df_2022['pfx_x'] = df_2022['pfx_x']*12
df_2022['pfx_z'] = df_2022['pfx_z']*12
df_2023['pfx_x'] = df_2023['pfx_x']*12
df_2023['pfx_z'] = df_2023['pfx_z']*12


#multiply pfx_x by -1 to get pitchers view
df_2022['pfx_x'] = df_2022['pfx_x']*-1
df_2023['pfx_x'] = df_2023['pfx_x']*-1

df_2022.rename(columns={'game_date': 'Date', 'player_name':'Pitcher','p_throws':'Hand','pitch_type':'TaggedPitchType','release_speed':'RelSpeed','release_pos_x':'RelSide','release_pos_z':'RelHeight','pfx_x':'HorzBreak','pfx_z':'InducedVertBreak','release_extension':'Extension','plate_x': 'PlateLocSide','plate_z': 'PlateLocHeight', 'release_spin_rate':'SpinRate'},inplace=True)
df_2023.rename(columns={'game_date': 'Date', 'player_name':'Pitcher','p_throws':'Hand','pitch_type':'TaggedPitchType','release_speed':'RelSpeed','release_pos_x':'RelSide','release_pos_z':'RelHeight','pfx_x':'HorzBreak','pfx_z':'InducedVertBreak','release_extension':'Extension','plate_x': 'PlateLocSide','plate_z': 'PlateLocHeight','release_spin_rate':'SpinRate'},inplace=True)

In [483]:
#if Event is == NaN fill with description
df_2022['events'].fillna(df_2022['description'], inplace=True)
df_2023['events'].fillna(df_2023['description'], inplace=True)

In [484]:
#df_2023 Pitcher	TaggedPitchType	RelSpeed	release_spin_rate	InducedVertBreak	HorzBreak	RelSide	RelHeight	Extension	
df_2022 = df_2022[['Pitcher','Hand','TaggedPitchType','events','description','RelSpeed','SpinRate','InducedVertBreak','HorzBreak','RelSide','RelHeight','Extension','PlateLocSide','PlateLocHeight','batter','launch_speed','launch_angle','hit_distance_sc','delta_run_exp']]
df_2023 = df_2023[['Pitcher','Hand','TaggedPitchType','events','description','RelSpeed','SpinRate','InducedVertBreak','HorzBreak','RelSide','RelHeight','Extension','PlateLocSide','PlateLocHeight','batter','launch_speed','launch_angle','hit_distance_sc','delta_run_exp']]

In [485]:
#run values 
field_out = -0.1955687665555
force_out = -0.1955687665555
other_out = -0.1955687665555
fielders_choice_out = -0.1955687665555
called_strike = -0.118124935770601
swinging_strike = -0.118124935770601
ball = 0.0636883289483747
foul = -0.0380502742575014
single = 0.467292970729251
double = 0.766083122898271
triple = 1.05755624961515
home_run = 1.374328827219
strikeout = -0.118124935770601
fielders_choice = -0.1955687665555
hit_by_pitch = 0.0636883289483747
walk = 0.0636883289483747
field_error = -0.1955687665555
walk = 0.0636883289483747
sac_fly = 0.0636883289483747
double_play = -0.1955687665555
wild_pitch =0.0636883289483747
blocked_ball = 0.0636883289483747
grounded_into_double_play = -0.1955687665555
foul_bunt = -0.0380502742575014
foul_tip = -0.0380502742575014
sac_bunt_double_play = -0.1955687665555
swinging_strike_blocked = -0.118124935770601
missed_bunt = -0.118124935770601
sac_bunt = 0.0636883289483747
pitchout = 0
caught_stealing_2b = 0
bunt_foul_tip = -0.0380502742575014
strikeout_double_play = -0.118124935770601
pickoff_3b = 0 
catcher_interf = 0
caught_stealing_3b = 0 
pickoff_caught_stealing_2b = 0
triple_play = -0.118124935770601
caught_stealing_home = 0 
sac_fly_double_play = -0.1955687665555
pickoff_1b = 0 
pickoff_caught_stealing_home = 0
pickoff_caught_stealing_3b = 0 
game_advisory = 0
pickoff_2b = 0

In [486]:
df_2022['RunValue'] = df_2022['events'].map({'field_out':field_out,'force_out':force_out,'other_out':other_out,'fielders_choice_out':fielders_choice_out,'called_strike':called_strike,'swinging_strike':swinging_strike,'ball':ball,'foul':foul,'single':single,'double':double,'triple':triple,'home_run':home_run,'strikeout':strikeout,'fielders_choice':fielders_choice,'hit_by_pitch':hit_by_pitch,'walk':walk,'field_error':field_error,'walk':walk,'sac_fly':sac_fly,'double_play':double_play,'wild_pitch':wild_pitch,'blocked_ball':blocked_ball,'grounded_into_double_play':grounded_into_double_play,'foul_bunt':foul_bunt,'foul_tip':foul_tip,'sac_bunt_double_play':sac_bunt_double_play,'swinging_strike_blocked':swinging_strike_blocked,'missed_bunt':missed_bunt,'sac_bunt':sac_bunt,'pitchout':pitchout,'caught_stealing_2b':caught_stealing_2b,'bunt_foul_tip':bunt_foul_tip,'strikeout_double_play':strikeout_double_play,'pickoff_3b':pickoff_3b,'catcher_interf':catcher_interf,'caught_stealing_3b':caught_stealing_3b,'pickoff_caught_stealing_2b':pickoff_caught_stealing_2b,'triple_play':triple_play,'caught_stealing_home':caught_stealing_home,'sac_fly_double_play':sac_fly_double_play,'pickoff_1b':pickoff_1b,'pickoff_caught_stealing_home':pickoff_caught_stealing_home,'pickoff_caught_stealing_3b':pickoff_caught_stealing_3b,'game_advisory':game_advisory,'pickoff_2b':pickoff_2b})
df_2023['RunValue'] = df_2023['events'].map({'field_out':field_out,'force_out':force_out,'other_out':other_out,'fielders_choice_out':fielders_choice_out,'called_strike':called_strike,'swinging_strike':swinging_strike,'ball':ball,'foul':foul,'single':single,'double':double,'triple':triple,'home_run':home_run,'strikeout':strikeout,'fielders_choice':fielders_choice,'hit_by_pitch':hit_by_pitch,'walk':walk,'field_error':field_error,'walk':walk,'sac_fly':sac_fly,'double_play':double_play,'wild_pitch':wild_pitch,'blocked_ball':blocked_ball,'grounded_into_double_play':grounded_into_double_play,'foul_bunt':foul_bunt,'foul_tip':foul_tip,'sac_bunt_double_play':sac_bunt_double_play,'swinging_strike_blocked':swinging_strike_blocked,'missed_bunt':missed_bunt,'sac_bunt':sac_bunt,'pitchout':pitchout,'caught_stealing_2b':caught_stealing_2b,'bunt_foul_tip':bunt_foul_tip,'strikeout_double_play':strikeout_double_play,'pickoff_3b':pickoff_3b,'catcher_interf':catcher_interf,'caught_stealing_3b':caught_stealing_3b,'pickoff_caught_stealing_2b':pickoff_caught_stealing_2b,'triple_play':triple_play,'caught_stealing_home':caught_stealing_home,'sac_fly_double_play':sac_fly_double_play,'pickoff_1b':pickoff_1b,'pickoff_caught_stealing_home':pickoff_caught_stealing_home,'pickoff_caught_stealing_3b':pickoff_caught_stealing_3b,'game_advisory':game_advisory,'pickoff_2b':pickoff_2b})


In [487]:
#replace all Nan Events with Descriptions
df_2022['events'].fillna(df_2022['description'], inplace=True)
df_2023['events'].fillna(df_2023['description'], inplace=True)


In [488]:
#filter only balls in play == hit_into_play
df_2022_hit_into_play = df_2022[df_2022['description'] == 'hit_into_play']
df_2023_hit_into_play = df_2023[df_2023['description'] == 'hit_into_play']

In [489]:
df_not_hit_into_play = df_2022[df_2022['description'] != 'hit_into_play']
df_not_hit_into_play_2023 = df_2023[df_2023['description'] != 'hit_into_play']

In [490]:
#predict run value based on launch_speed','launch_angle','hit_distanc_sc'
X = df_2022_hit_into_play[['launch_speed','launch_angle','hit_distance_sc']]
y = df_2022_hit_into_play['RunValue']
X1 = df_2023_hit_into_play[['launch_speed','launch_angle','hit_distance_sc']]
y1 = df_2023_hit_into_play['RunValue']

In [491]:
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=42)
X1_train, X1_test, y1_train, y1_test =  train_test_split(X1, y1, test_size=0.2, random_state=42)

In [492]:
from sklearn.model_selection import train_test_split

# Assuming you have already split into X_train, y_train:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(X1_train, y1_train, test_size=0.2, random_state=42)

In [ ]:
params = {
    'max_depth': 5,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'alpha': 0.05,
    'lambda': 0.5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror'
}

xgb_model = XGBRegressor(**params)
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)
xgb_model_1 = XGBRegressor(**params)
xgb_model_1.fit(X1_train, y1_train, eval_set=[(X_val_1, y_val_1)], early_stopping_rounds=10, verbose=False)

In [494]:
from sklearn.utils import resample
from sklearn.metrics import r2_score

n_estimators = 10  # Number of XGBoost models in the ensemble
predictions = []

# 1. Create bootstrap samples and train XGBoost models
for i in range(n_estimators):
    X_train_sample, y_train_sample = resample(X_train, y_train)
    model = XGBRegressor(**params)
    model.fit(X_train_sample, y_train_sample, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)
    
    # 2. Predict on validation set
    preds = model.predict(X_val)
    predictions.append(preds)

# 3. Average predictions from all models for ensemble prediction
ensemble_preds = np.mean(predictions, axis=0)

# Calculate R^2 for ensemble prediction
ensemble_r2 = r2_score(y_val, ensemble_preds)

In [495]:
ensemble_r2

0.49774610651139495

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)

# Perform cross-validation
cv_results = xgb.cv(
    dtrain=dtrain, 
    params=params, 
    nfold=5, 
    num_boost_round=200, 
    early_stopping_rounds=10, 
    metrics="rmse",
    as_pandas=True, 
    seed=42
)

In [607]:
#Variance 
variance_y = y_train.var()
best_r_squared = 1 - (cv_results['test-rmse-mean'].min() ** 2) / variance_y
#print best r^2 without using scientific notation
print('xRun Value EV/LA Best  R^2:', '{:.20f}'.format(best_r_squared))


xRun Value EV/LA Best  R^2: -0.61678188005002909655


In [ ]:
df_2022_hit_into_play['RunValue'] = xgb_model.predict(df_2022_hit_into_play[['launch_speed','launch_angle','hit_distance_sc']])
df_2023_hit_into_play['RunValue'] = xgb_model_1.predict(df_2023_hit_into_play[['launch_speed','launch_angle','hit_distance_sc']])

In [501]:
#combine df_2022_hit_into_play and df_not_hit_into_play 
df_2022_1 = pd.concat([df_2022_hit_into_play,df_not_hit_into_play ],ignore_index=True)
df_2023_1 = pd.concat([df_2023_hit_into_play,df_not_hit_into_play_2023 ],ignore_index=True)


In [502]:
df_2022 = df_2022_1
df_2023 = df_2023_1

In [ ]:
df_2023['TaggedPitchType'].unique()

In [504]:
#remove any Nan RunValue
df_2022 = df_2022[df_2022['RunValue'].notna()]
df_2023 = df_2023[df_2023['RunValue'].notna()]

In [ ]:
#change pitch type to match trackman data
#FF, SI, FS,FA  = Fastball
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['FF','FA'],'Fastball')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['FF','FA'],'Fastball')

df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['SI'], 'Sinker')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['SI'], 'Sinker')

#SL,ST,FC  = Slider
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['SL'],'Slider')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['SL'],'Slider')

df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['ST'],'Sweeper')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['ST'],'Sweeper')

#CH, EP, SC, FO = ChangeUp
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['CH','EP','SC','FO','KN'],'ChangeUp')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['CH','EP','SC','FO','KN'],'ChangeUp')
#SV, CU, KC = Curveball
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['SV','CU','KC'],'Curveball')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['SV','CU','KC'],'Curveball')
#'FC' = Cutter
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['FC'],'Cutter')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['FC'],'Cutter')
#SPL = FS
df_2022['TaggedPitchType'] = df_2022['TaggedPitchType'].replace(['FS'],'Splitter')
df_2023['TaggedPitchType'] = df_2023['TaggedPitchType'].replace(['FS'],'Splitter')

In [ ]:
df_2022['ABS_Horizontal'] = abs(df_2022['HorzBreak'])
df_2023['ABS_Horizontal'] = abs(df_2023['HorzBreak'])
df_2022['ABS_RelSide'] = abs(df_2022['RelSide'])
df_2023['ABS_RelSide'] = abs(df_2023['RelSide'])
df_2022['differential_break'] = abs(df_2022['InducedVertBreak'] - df_2022['ABS_Horizontal'])
df_2023['differential_break'] = abs(df_2023['InducedVertBreak'] - df_2023['ABS_Horizontal'])

In [507]:
#fastballs = 'Fastball', 'Sinker', 'TwoSeamFastBall', 'FourSeamFastBall', 'OneSeamFastBall'
dfb2 = df_2022[df_2022.TaggedPitchType.isin(['Fastball', 'FourSeamFastball', 'OneSeamFastBall'])]
dfb3 = df_2023[df_2023.TaggedPitchType.isin(['Fastball', 'FourSeamFastball', 'OneSeamFastBall'])]
#'Sinker', 'TwoSeamFastBall',
dsi2 = df_2022[df_2022.TaggedPitchType.isin(['Sinker', 'TwoSeamFastBall'])]
dsi3 = df_2023[df_2023.TaggedPitchType.isin(['Sinker', 'TwoSeamFastBall'])]
#sliders = 'Slider', 'Cutter'
dsl2 = df_2022[df_2022.TaggedPitchType.isin(['Slider'])]
dsl3 = df_2023[df_2023.TaggedPitchType.isin(['Slider'])]
#Sweeper
dst2 = df_2022[df_2022.TaggedPitchType.isin(['Sweeper'])]
dst3 = df_2023[df_2023.TaggedPitchType.isin(['Sweeper'])]
#curveballs = 'Curveball', 'KnuckleCurve'
dcb2 = df_2022[df_2022.TaggedPitchType.isin(['Curveball', 'KnuckleCurve'])]
dcb3 = df_2023[df_2023.TaggedPitchType.isin(['Curveball', 'KnuckleCurve'])]
#changeups = 'Changeup', 'Splitter', 'Forkball', 'Screwball'
dch2 = df_2022[df_2022.TaggedPitchType.isin(['ChangeUp'])]
dch3 = df_2023[df_2023.TaggedPitchType.isin(['ChangeUp'])]
#cutters = 'Cutter'
dct2 = df_2022[df_2022.TaggedPitchType.isin(['Cutter'])]
dct3 = df_2023[df_2023.TaggedPitchType.isin(['Cutter'])]
#splitter
dsp2 = df_2022[df_2022.TaggedPitchType.isin(['Splitter'])]
dsp3 = df_2023[df_2023.TaggedPitchType.isin(['Splitter'])]

In [510]:
#delete all rows with nan values
dfb2 = dfb2.dropna()
dsi2 = dsi2.dropna()
dsl2 = dsl2.dropna()
dst2 = dst2.dropna()
dcb2 = dcb2.dropna()
dch2 = dch2.dropna()
dct2 = dct2.dropna()
dsp2 = dsp2.dropna()

In [511]:
#xRV stuff
X = dfb2[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']]
y = dfb2['RunValue']
X1 = dsi2[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']]
y1 = dsi2['RunValue']
X2 = dsl2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y2 = dsl2['RunValue']
X3 = dst2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y3 = dst2['RunValue']
X4 = dcb2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y4 = dcb2['RunValue']
X5 = dch2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y5 = dch2['RunValue']
X6 = dct2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y6 = dct2['RunValue']
X7 = dsp2[['RelSpeed','SpinRate','InducedVertBreak','ABS_Horizontal','RelHeight', 'ABS_RelSide', 'Extension']]
y7 = dsp2['RunValue']

In [512]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.25, random_state=101)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.25, random_state=101)

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.25, random_state=101)

X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.25, random_state=101)

X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, test_size=0.25, random_state=101)

X6_train, X6_test, y6_train, y6_test = train_test_split(X6, y6, test_size=0.25, random_state=101)

X7_train, X7_test, y7_train, y7_test = train_test_split(X7, y7, test_size=0.25, random_state=101)


In [ ]:
params = {
    'max_depth': 5,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'alpha': 0.05,
    'lambda': 0.5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror'
}

xgb_model_0 = XGBRegressor(**params)
xgb_model_0.fit(X_train, y_train)

#sinker
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_train, y1_train, test_size=0.2, random_state=42)
params = {
    'max_depth': 5,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'alpha': 0.05,
    'lambda': 0.5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror'
}

xgb_model1 = XGBRegressor(**params)
xgb_model1.fit(X1_train, y1_train)

#cslider
X2_train, X2_val, y2_train, y2_val = train_test_split(X2_train, y2_train, test_size=0.2, random_state=42)
params = {
    'max_depth': 5,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'alpha': 0.05,
    'lambda': 0.5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror'
}
xgb_model_2 = XGBRegressor(**params)
xgb_model_2.fit(X2_train, y2_train)

#csweeper
X3_train, X3_val, y3_train, y3_val = train_test_split(X3_train, y3_train, test_size=0.2, random_state=42)
params = {
    'max_depth': 5,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'alpha': 0.05,
    'lambda': 0.5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror'
}
xgb_model_3 = XGBRegressor(**params)
xgb_model_3.fit(X3_train, y3_train)

#curveball
X4_train, X4_val, y4_train, y4_val = train_test_split(X4_train, y4_train, test_size=0.2, random_state=42)
params = {
    'max_depth': 5, 
    'n_estimators': 100,
    'learning_rate': 0.1,
    'alpha': 0.05,
    'lambda': 0.5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror'
}
xgb_model_4 = XGBRegressor(**params)
xgb_model_4.fit(X4_train, y4_train)

#changeup 
X5_train, X5_val, y5_train, y5_val = train_test_split(X5_train, y5_train, test_size=0.2, random_state=42)
params = {
    'max_depth': 5, 
    'n_estimators': 100,
    'learning_rate': 0.1,
    'alpha': 0.05,
    'lambda': 0.5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror'
}
xgb_model_5 = XGBRegressor(**params)
xgb_model_5.fit(X5_train, y5_train)

#cutter
X6_train, X6_val, y6_train, y6_val = train_test_split(X6_train, y6_train, test_size=0.2, random_state=42)
params = {
    'max_depth': 5, 
    'n_estimators': 100,
    'learning_rate': 0.1,
    'alpha': 0.05,
    'lambda': 0.5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror'
}
xgb_model_6 = XGBRegressor(**params)
xgb_model_6.fit(X6_train, y6_train)

#splitter
X7_train, X7_val, y7_train, y7_val = train_test_split(X7_train, y7_train, test_size=0.2, random_state=42)
params = {
    'max_depth': 5, 
    'n_estimators': 100,
    'learning_rate': 0.1,
    'alpha': 0.05,
    'lambda': 0.5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror'
}
xgb_model_7 = XGBRegressor(**params)
xgb_model_7.fit(X7_train, y7_train)

In [517]:
dfb2['xRV_xgb']= xgb_model_0.predict(dfb2[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']])
dsi2['xRV_xgb']= xgb_model1.predict(dsi2[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']])
dsl2['xRV_xgb']= xgb_model_2.predict(dsl2[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])
dst2['xRV_xgb']= xgb_model_3.predict(dst2[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])
dcb2['xRV_xgb']= xgb_model_4.predict(dcb2[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])
dch2['xRV_xgb']= xgb_model_5.predict(dch2[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])
dct2['xRV_xgb']= xgb_model_6.predict(dct2[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])
dsp2['xRV_xgb']= xgb_model_7.predict(dsp2[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])

In [ ]:
dfb3['xRV_xgb']= xgb_model_0.predict(dfb3[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']])
dsi3['xRV_xgb']= xgb_model1.predict(dsi3[['RelSpeed','SpinRate','differential_break','RelHeight', 'ABS_RelSide', 'Extension']])
dsl3['xRV_xgb']= xgb_model_2.predict(dsl3[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])
dst3['xRV_xgb']= xgb_model_3.predict(dst3[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])
dcb3['xRV_xgb']= xgb_model_4.predict(dcb3[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])
dch3['xRV_xgb']= xgb_model_5.predict(dch3[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])
dct3['xRV_xgb']= xgb_model_6.predict(dct3[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])
dsp3['xRV_xgb']= xgb_model_7.predict(dsp3[['RelSpeed', 'SpinRate','InducedVertBreak', 'ABS_Horizontal', 'RelHeight', 'ABS_RelSide', 'Extension']])

In [520]:
#find the r^2 for all the variables
#create the FB expected whiff rate
X = dfb2[['xRV_xgb']]
y = dfb2['RunValue']
X2 = dsi2[['xRV_xgb']]
y2 = dsi2['RunValue']
X3 = dsl2[['xRV_xgb']]
y3 = dsl2['RunValue']
X4 = dst2[['xRV_xgb']]
y4 = dst2['RunValue']
X5 = dcb2[['xRV_xgb']]
y5 = dcb2['RunValue']
X6 = dch2[['xRV_xgb']]
y6 = dch2['RunValue']
X7 = dct2[['xRV_xgb']]
y7 = dct2['RunValue']
X8 = dsp2[['xRV_xgb']]
y8 = dsp2['RunValue']

In [521]:
dfb2_max = dfb2['xRV_xgb'].max()
dsi2_max = dsi2['xRV_xgb'].max()
dsl2_max = dsl2['xRV_xgb'].max()
dst2_max = dst2['xRV_xgb'].max()
dcb2_max = dcb2['xRV_xgb'].max()
dch2_max = dch2['xRV_xgb'].max()
dct2_max = dct2['xRV_xgb'].max()
dsp2_max = dsp2['xRV_xgb'].max()
dfb3_max = dfb3['xRV_xgb'].max()
dsi3_max = dsi3['xRV_xgb'].max()
dsl3_max = dsl3['xRV_xgb'].max()
dst3_max = dst3['xRV_xgb'].max()
dcb3_max = dcb3['xRV_xgb'].max()
dch3_max = dch3['xRV_xgb'].max()
dct3_max = dct3['xRV_xgb'].max()
dsp3_max = dsp3['xRV_xgb'].max()

In [ ]:
#scaled 
dfb2['xRV_Scaled'] = dfb2['xRV_xgb'] - dfb2_max
dfb2['xRV/100_stuff_scaled_abs'] = abs(dfb2['xRV_Scaled'])
dfb2['Stuff_plus'] = dfb2['xRV/100_stuff_scaled_abs'] / dfb2['xRV/100_stuff_scaled_abs'].mean() * 100
dfb3['xRV_Scaled'] = dfb3['xRV_xgb'] - dfb3_max
dfb3['xRV/100_stuff_scaled_abs'] = abs(dfb3['xRV_Scaled'])
dfb3['Stuff_plus'] = dfb3['xRV/100_stuff_scaled_abs'] / dfb3['xRV/100_stuff_scaled_abs'].mean() * 100

#scaled 
dsi2['xRV_Scaled'] = dsi2['xRV_xgb'] - dsi2_max
dsi2['xRV/100_stuff_scaled_abs'] = abs(dsi2['xRV_Scaled'])
dsi2['Stuff_plus'] = dsi2['xRV/100_stuff_scaled_abs'] / dsi2['xRV/100_stuff_scaled_abs'].mean() * 100
dsi3['xRV_Scaled'] = dsi3['xRV_xgb'] - dsi3_max
dsi3['xRV/100_stuff_scaled_abs'] = abs(dsi3['xRV_Scaled'])
dsi3['Stuff_plus'] = dsi3['xRV/100_stuff_scaled_abs'] / dsi3['xRV/100_stuff_scaled_abs'].mean() * 100

#scaled 
dsl2['xRV_Scaled'] = dsl2['xRV_xgb'] - dfb2_max
dsl2['xRV/100_stuff_scaled_abs'] = abs(dsl2['xRV_Scaled'])
dsl2['Stuff_plus'] = dsl2['xRV/100_stuff_scaled_abs'] / dsl2['xRV/100_stuff_scaled_abs'].mean() * 100
dsl3['xRV_Scaled'] = dsl3['xRV_xgb'] - dsl3_max
dsl3['xRV/100_stuff_scaled_abs'] = abs(dsl3['xRV_Scaled'])
dsl3['Stuff_plus'] = dsl3['xRV/100_stuff_scaled_abs'] / dsl3['xRV/100_stuff_scaled_abs'].mean() * 100

#scaled 
dst2['xRV_Scaled'] = dst2['xRV_xgb'] - dst2_max
dst2['xRV/100_stuff_scaled_abs'] = abs(dst2['xRV_Scaled'])
dst2['Stuff_plus'] = dst2['xRV/100_stuff_scaled_abs'] / dst2['xRV/100_stuff_scaled_abs'].mean() * 100
dst3['xRV_Scaled'] = dst3['xRV_xgb'] - dst3_max
dst3['xRV/100_stuff_scaled_abs'] = abs(dst3['xRV_Scaled'])
dst3['Stuff_plus'] = dst3['xRV/100_stuff_scaled_abs'] / dst3['xRV/100_stuff_scaled_abs'].mean() * 100

#scaled 
dcb2['xRV_Scaled'] = dcb2['xRV_xgb'] - dcb2_max
dcb2['xRV/100_stuff_scaled_abs'] = abs(dcb2['xRV_Scaled'])
dcb2['Stuff_plus'] = dcb2['xRV/100_stuff_scaled_abs'] / dcb2['xRV/100_stuff_scaled_abs'].mean() * 100
dcb3['xRV_Scaled'] = dcb3['xRV_xgb'] - dcb3_max
dcb3['xRV/100_stuff_scaled_abs'] = abs(dcb3['xRV_Scaled'])
dcb3['Stuff_plus'] = dcb3['xRV/100_stuff_scaled_abs'] / dcb3['xRV/100_stuff_scaled_abs'].mean() * 100

#scaled 
dch2['xRV_Scaled'] = dch2['xRV_xgb'] - dch2_max
dch2['xRV/100_stuff_scaled_abs'] = abs(dch2['xRV_Scaled'])
dch2['Stuff_plus'] = dch2['xRV/100_stuff_scaled_abs'] / dch2['xRV/100_stuff_scaled_abs'].mean() * 100
dch3['xRV_Scaled'] = dch3['xRV_xgb'] - dch3_max
dch3['xRV/100_stuff_scaled_abs'] = abs(dch3['xRV_Scaled'])
dch3['Stuff_plus'] = dch3['xRV/100_stuff_scaled_abs'] / dch3['xRV/100_stuff_scaled_abs'].mean() * 100

#scaled 
dct2['xRV_Scaled'] = dct2['xRV_xgb'] - dct2_max
dct2['xRV/100_stuff_scaled_abs'] = abs(dct2['xRV_Scaled'])
dct2['Stuff_plus'] = dct2['xRV/100_stuff_scaled_abs'] / dct2['xRV/100_stuff_scaled_abs'].mean() * 100
dct3['xRV_Scaled'] = dct3['xRV_xgb'] - dct3_max
dct3['xRV/100_stuff_scaled_abs'] = abs(dct3['xRV_Scaled'])
dct3['Stuff_plus'] = dct3['xRV/100_stuff_scaled_abs'] / dct3['xRV/100_stuff_scaled_abs'].mean() * 100

#splitter
dsp2['xRV_Scaled'] = dsp2['xRV_xgb'] - dsp2_max
dsp2['xRV/100_stuff_scaled_abs'] = abs(dsp2['xRV_Scaled'])
dsp2['Stuff_plus'] = dsp2['xRV/100_stuff_scaled_abs'] / dsp2['xRV/100_stuff_scaled_abs'].mean() * 100
dsp3['xRV_Scaled'] = dsp3['xRV_xgb'] - dsp3_max
dsp3['xRV/100_stuff_scaled_abs'] = abs(dsp3['xRV_Scaled'])
dsp3['Stuff_plus'] = dsp3['xRV/100_stuff_scaled_abs'] / dsp3['xRV/100_stuff_scaled_abs'].mean() * 100


In [ ]:


# Assuming df_total is your DataFrame and 'Stuff_plus' is the name of your column
current_std_fb = dfb3['Stuff_plus'].std()

# Determine the scaling factor needed to adjust the standard deviation to 10
desired_std = 10
scaling_factor = desired_std / current_std_fb

# Apply the scaling factor to each value in the column
# Subtract the mean, scale the zero-mean data, and then add the original mean back
mean = dfb3['Stuff_plus'].mean()
dfb3['Final_Adjusted_Stuff_Plus'] = ((dfb3['Stuff_plus'] - mean) * scaling_factor) + mean

# Verify the transformation by calculating the new standard deviation and mean
new_std = dfb3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dfb3['Final_Adjusted_Stuff_Plus'].mean()

current_std_si = dsi3['Stuff_plus'].std()

# Determine the scaling factor needed to adjust the standard deviation to 10
desired_std = 10
scaling_factor = desired_std / current_std_si

# Apply the scaling factor to each value in the column
# Subtract the mean, scale the zero-mean data, and then add the original mean back
mean = dsi3['Stuff_plus'].mean()
dsi3['Final_Adjusted_Stuff_Plus'] = ((dsi3['Stuff_plus'] - mean) * scaling_factor) + mean

# Verify the transformation by calculating the new standard deviation and mean
new_std = dsi3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dsi3['Final_Adjusted_Stuff_Plus'].mean()

current_std_sl = dsl3['Stuff_plus'].std()

# Determine the scaling factor needed to adjust the standard deviation to 10
desired_std = 10
scaling_factor = desired_std / current_std_sl

# Apply the scaling factor to each value in the column
# Subtract the mean, scale the zero-mean data, and then add the original mean back
mean = dsl3['Stuff_plus'].mean()
dsl3['Final_Adjusted_Stuff_Plus'] = ((dsl3['Stuff_plus'] - mean) * scaling_factor) + mean

# Verify the transformation by calculating the new standard deviation and mean
new_std = dsl3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dsl3['Final_Adjusted_Stuff_Plus'].mean()

current_std_st = dst3['Stuff_plus'].std()

# Determine the scaling factor needed to adjust the standard deviation to 10
desired_std = 10
scaling_factor = desired_std / current_std_st

# Apply the scaling factor to each value in the column
# Subtract the mean, scale the zero-mean data, and then add the original mean back
mean = dst3['Stuff_plus'].mean()
dst3['Final_Adjusted_Stuff_Plus'] = ((dst3['Stuff_plus'] - mean) * scaling_factor) + mean

# Verify the transformation by calculating the new standard deviation and mean
new_std = dst3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dst3['Final_Adjusted_Stuff_Plus'].mean()

current_std_cb = dcb3['Stuff_plus'].std()

# Determine the scaling factor needed to adjust the standard deviation to 10
desired_std = 10
scaling_factor = desired_std / current_std_cb

# Apply the scaling factor to each value in the column
# Subtract the mean, scale the zero-mean data, and then add the original mean back
mean = dcb3['Stuff_plus'].mean()
dcb3['Final_Adjusted_Stuff_Plus'] = ((dcb3['Stuff_plus'] - mean) * scaling_factor) + mean

# Verify the transformation by calculating the new standard deviation and mean
new_std = dcb3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dcb3['Final_Adjusted_Stuff_Plus'].mean()

current_std_ch = dch3['Stuff_plus'].std()

# Determine the scaling factor needed to adjust the standard deviation to 10
desired_std = 10
scaling_factor = desired_std / current_std_ch

# Apply the scaling factor to each value in the column
# Subtract the mean, scale the zero-mean data, and then add the original mean back
mean = dch3['Stuff_plus'].mean()
dch3['Final_Adjusted_Stuff_Plus'] = ((dch3['Stuff_plus'] - mean) * scaling_factor) + mean

# Verify the transformation by calculating the new standard deviation and mean
new_std = dch3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dch3['Final_Adjusted_Stuff_Plus'].mean()

current_std_ct = dct3['Stuff_plus'].std()

# Determine the scaling factor needed to adjust the standard deviation to 10
desired_std = 10
scaling_factor = desired_std / current_std_ch

# Apply the scaling factor to each value in the column
# Subtract the mean, scale the zero-mean data, and then add the original mean back
mean = dct3['Stuff_plus'].mean()
dct3['Final_Adjusted_Stuff_Plus'] = ((dct3['Stuff_plus'] - mean) * scaling_factor) + mean

# Verify the transformation by calculating the new standard deviation and mean
new_std = dct3['Final_Adjusted_Stuff_Plus'].std()
new_mean = dct3['Final_Adjusted_Stuff_Plus'].mean()

#splitter
current_std_sp = dsp3['Stuff_plus'].std()

# Determine the scaling factor needed to adjust the standard deviation to 10
desired_std = 10
scaling_factor = desired_std / current_std_sp

# Apply the scaling factor to each value in the column
# Subtract the mean, scale the zero-mean data, and then add the original mean back
mean = dsp3['Stuff_plus'].mean()
dsp3['Final_Adjusted_Stuff_Plus'] = ((dsp3['Stuff_plus'] - mean) * scaling_factor) + mean




In [559]:
#combine all the dataframes
df_2022 = pd.concat([dfb2, dsi2, dsl2, dst2, dcb2, dch2, dct2], ignore_index=True)
df_2023_total = pd.concat([dfb3, dsi3, dsl3, dst3, dcb3, dch3, dct3], ignore_index=True)

In [560]:
#groupby pitcher and TaggedPitchType
df_2023_total_pitch_RV = df_2023_total.groupby(['Pitcher', 'TaggedPitchType']).agg({'RelSpeed': 'mean','InducedVertBreak':'mean','HorzBreak':'mean','Final_Adjusted_Stuff_Plus': 'mean', 'RunValue': 'sum', 'launch_speed':'mean' }).reset_index()
df_2023_total_pitcher_RV = df_2023_total.groupby(['Pitcher']).agg({'Final_Adjusted_Stuff_Plus': 'mean', 'RunValue': 'sum', 'launch_speed':'mean' }).reset_index()
df_2023_total_pitcher_RV['Count'] = df_2023_total.groupby(['Pitcher']).size().reset_index(name='Count')['Count']

In [606]:
from sklearn.linear_model import LinearRegression
#calculate the r^2 of Final_Adjusted_Stuff_Plus on RunValue
X = df_2023_total_pitcher_RV[['Final_Adjusted_Stuff_Plus']]
y = df_2023_total_pitcher_RV['RunValue']
#linear regression
lm = LinearRegression()
lm.fit(X, y)
#predict
y_pred = lm.predict(X)
#r^2
r2 = r2_score(y, y_pred)
print('Unweighted Stuff+ R^2:', r2)

Unweighted Stuff+ R^2: 0.09287529754079704


In [562]:
df_player = pd.read_csv('/Users/johndavis/Desktop/PitcherTeam.csv')
df_player.rename(columns = {'player_name':'Pitcher', 'PitcherTeam':'Team'}, inplace = True)

In [563]:
df_2023_total['Team'] = df_2023_total['Pitcher'].map(df_player.set_index('Pitcher')['Team'])
df_2023_total_pitch_RV['Team'] = df_2023_total_pitch_RV['Pitcher'].map(df_player.set_index('Pitcher')['Team'])

In [564]:
df_2023_total_pitcher_RV.rename(columns = {'Pitcher':'Name'}, inplace = True)

In [565]:
df_ERA = pd.read_csv('/Users/johndavis/Desktop/2023_ERA.csv')

In [568]:
#flip names so they match first and last name instead of last, first
df_2023_total_pitcher_RV['Name'] = df_2023_total_pitcher_RV['Name'].str.split(', ').str[::-1].str.join(' ')

,Final_Adjusted_Stuff_Plus,RunValue,launch_speed,Count
Final_Adjusted_Stuff_Plus,1.000000,-0.304754,-0.184984,0.161421
RunValue,-0.304754,1.000000,0.207878,-0.155659
launch_speed,-0.184984,0.207878,1.000000,-0.095513
Count,0.161421,-0.155659,-0.095513,1.000000


In [569]:
#combine the ERA and the RunValue
df_2023_total_pitcher_RV_ERA = df_2023_total_pitcher_RV.merge(df_ERA, on='Name')

In [571]:
#DELETE SO	uBB	HBP	HR	
del df_2023_total_pitcher_RV_ERA['SO']
del df_2023_total_pitcher_RV_ERA['uBB']
del df_2023_total_pitcher_RV_ERA['HBP']
del df_2023_total_pitcher_RV_ERA['HR']

In [590]:
df_2023_total_pitcher_RV_ERA_count_4 = df_2023_total_pitcher_RV_ERA[df_2023_total_pitcher_RV_ERA['Count'] > 1000]

In [599]:
#calculate the r^2 of Final_Adjusted_Stuff_Plus on RunValue
X = df_2023_total_pitcher_RV_ERA_count_1[['Final_Adjusted_Stuff_Plus']]
y = df_2023_total_pitcher_RV_ERA_count_1['RunValue']

#linear regression
lm = LinearRegression()
lm.fit(X, y)
#predict
y_pred = lm.predict(X)
#r^2
r2 = r2_score(y, y_pred)
print('Model Summary for 2023 Season')
print('*************')
print('Full Arsenal || 250 Pitch Minimum')
print('*************')
print('Stuff+ Model R^2:', r2)
#print the correlation between Final_Adjusted_Stuff_Plus and ERA / Final_Adjusted_Stuff_Plus and FIP / Final_Adjusted_Stuff_Plus and wOBA_against	
print('Descriptive Correlation between Stuff+ and ERA:', df_2023_total_pitcher_RV_ERA_count_1['Final_Adjusted_Stuff_Plus'].corr(df_2023_total_pitcher_RV_ERA_count_1['ERA']))
print('Descriptive Correlation between Stuff+ and FIP:', df_2023_total_pitcher_RV_ERA_count_1['Final_Adjusted_Stuff_Plus'].corr(df_2023_total_pitcher_RV_ERA_count_1['FIP']))
print('Descriptive Correlation between Stuff+ and wOBA_against:', df_2023_total_pitcher_RV_ERA_count_1['Final_Adjusted_Stuff_Plus'].corr(df_2023_total_pitcher_RV_ERA_count_1['wOBA_against']))

Model Summary for 2023 Season
*************
Full Arsenal || 250 Pitch Minimum
*************
Stuff+ Model R^2: 0.20912971769733735
Descriptive Correlation between Stuff+ and ERA: -0.34188257295111313
Descriptive Correlation between Stuff+ and FIP: -0.348328740889808
Descriptive Correlation between Stuff+ and wOBA_against: -0.3710634395432807


In [601]:
#calculate the r^2 of Final_Adjusted_Stuff_Plus on RunValue
X = df_2023_total_pitcher_RV_ERA_count[['Final_Adjusted_Stuff_Plus']]
y = df_2023_total_pitcher_RV_ERA_count['RunValue']

#linear regression
lm = LinearRegression()
lm.fit(X, y)
#predict
y_pred = lm.predict(X)
#r^2
r2 = r2_score(y, y_pred)
print('Model Summary for 2023 Season')
print('*************')
print('Full Arsenal || 500 Pitch Minimum')
print('*************')
print('Stuff+ Model R^2:', r2)
#print the correlation between Final_Adjusted_Stuff_Plus and ERA / Final_Adjusted_Stuff_Plus and FIP / Final_Adjusted_Stuff_Plus and wOBA_against	
print('Descriptive Correlation between Stuff+ and ERA:', df_2023_total_pitcher_RV_ERA_count['Final_Adjusted_Stuff_Plus'].corr(df_2023_total_pitcher_RV_ERA_count['ERA']))
print('Descriptive Correlation between Stuff+ and FIP:', df_2023_total_pitcher_RV_ERA_count['Final_Adjusted_Stuff_Plus'].corr(df_2023_total_pitcher_RV_ERA_count['FIP']))
print('Descriptive Correlation between Stuff+ and wOBA_against:', df_2023_total_pitcher_RV_ERA_count['Final_Adjusted_Stuff_Plus'].corr(df_2023_total_pitcher_RV_ERA_count['wOBA_against']))

Model Summary for 2023 Season
*************
Full Arsenal || 500 Pitch Minimum
*************
Stuff+ Model R^2: 0.22952074798863087
Descriptive Correlation between Stuff+ and ERA: -0.3938126853115962
Descriptive Correlation between Stuff+ and FIP: -0.3945330323447696
Descriptive Correlation between Stuff+ and wOBA_against: -0.4282581540433598


In [602]:
#calculate the r^2 of Final_Adjusted_Stuff_Plus on RunValue
X = df_2023_total_pitcher_RV_ERA_count_4[['Final_Adjusted_Stuff_Plus']]
y = df_2023_total_pitcher_RV_ERA_count_4['RunValue']

#linear regression
lm = LinearRegression()
lm.fit(X, y)
#predict
y_pred = lm.predict(X)
#r^2
r2 = r2_score(y, y_pred)
print('Model Summary for 2023 Season')
print('*************')
print('Full Arsenal || 1000 Pitch Minimum')
print('*************')
print('Stuff+ Model R^2:', r2)
#print the correlation between Final_Adjusted_Stuff_Plus and ERA / Final_Adjusted_Stuff_Plus and FIP / Final_Adjusted_Stuff_Plus and wOBA_against	
print('Descriptive Correlation between Stuff+ and ERA:', df_2023_total_pitcher_RV_ERA_count_4['Final_Adjusted_Stuff_Plus'].corr(df_2023_total_pitcher_RV_ERA_count_4['ERA']))
print('Descriptive Correlation between Stuff+ and FIP:', df_2023_total_pitcher_RV_ERA_count_4['Final_Adjusted_Stuff_Plus'].corr(df_2023_total_pitcher_RV_ERA_count_4['FIP']))
print('Descriptive Correlation between Stuff+ and wOBA_against:', df_2023_total_pitcher_RV_ERA_count_4['Final_Adjusted_Stuff_Plus'].corr(df_2023_total_pitcher_RV_ERA_count_4['wOBA_against']))

Model Summary for 2023 Season
*************
Full Arsenal || 1000 Pitch Minimum
*************
Stuff+ Model R^2: 0.258439850165034
Descriptive Correlation between Stuff+ and ERA: -0.43462576397025826
Descriptive Correlation between Stuff+ and FIP: -0.48210312243510767
Descriptive Correlation between Stuff+ and wOBA_against: -0.48779034228651447
